# Explainable SVM for Medical Images


In [2]:
import os
import numpy as np
import cv2
import mlflow
import shap
import lime
import lime.lime_image
from skimage.segmentation import mark_boundaries
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mlflow.models.signature import infer_signature


In [ ]:
DATA_DIR = "/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray"  

def load_data(img_size=128):
    images, labels = [], []
    for label, folder in enumerate(['NORMAL', 'PNEUMONIA']):
        path = os.path.join(DATA_DIR, 'train', folder)
        for file in os.listdir(path):
            print(os.path.join(path,file))
            if '.jpeg' in file:
                img = cv2.imread(os.path.join(path, file), cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (img_size, img_size))
                images.append(img.flatten())
                labels.append(label)
    return np.array(images), np.array(labels)

def train_svm_model(X_train, y_train, kernel='linear'):
    model = make_pipeline(StandardScaler(), SVC(kernel=kernel, probability=True))
    model.fit(X_train, y_train)
    return model

def explain_with_lime(model, X, img_shape, index=0):
    explainer = lime.lime_image.LimeImageExplainer()
    def predict_fn(images):
        images = np.array([i.flatten() for i in images])
        return model.predict_proba(images)

    image = X[index].reshape(img_shape)
    explanation = explainer.explain_instance(image, predict_fn, top_labels=1, hide_color=0, num_samples=1000)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
    plt.imshow(mark_boundaries(temp, mask))
    plt.title("LIME Explanation")
    plt.axis('off')
    plt.show()

def explain_with_shap(model, X_sample):
    explainer = shap.Explainer(model.predict_proba, X_sample[:100])
    shap_values = explainer(X_sample[:10])
    shap.image_plot(shap_values)

In [9]:
mlflow.set_experiment("Medical SVM Pneumonia Detection")

X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

for kernel in ['linear', 'rbf', 'poly']:
    with mlflow.start_run(run_name=f"SVM_{kernel}"):
        model = train_svm_model(X_train, y_train, kernel)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        mlflow.log_param("kernel", kernel)
        mlflow.log_metric("accuracy", acc)
        mlflow.sklearn.log_model(model, f"svm_{kernel}_model")
        print(f"\n--- SVM ({kernel}) ---\n", classification_report(y_test, y_pred))

        explain_with_lime(model.named_steps['svc'], X_test, img_shape=(128, 128), index=0)
        
        try:
            explain_with_shap(model.named_steps['svc'], X_test)
        except Exception as e:
            print(f"SHAP explanation failed: {e}")

/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/IM-0499-0001-0002.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/IM-0584-0001.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/IM-0166-0001.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/IM-0429-0001.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/NORMAL2-IM-1002-0001.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/NORMAL2-IM-0846-0001.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/IM-0484-0001.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/IM-0629-0001-0001.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/NORMAL/NORMAL2-IM-0599-0001.jpeg
/home/alidb/Desktop/Masters/ML4P/project/DATA/chest_xray/chest_xray/train/N

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
